In [ ]:
import pandas as pd
import os
import pickle
import joblib
import numpy as np

from source.improved_transformer_forecasting import ImprovedTransformerForecasting, PositionalEncoding

#### 1. Config parameters

In [ ]:
# Dataset specific parameters
dataset_name = 'awsce-improved-3'                   # Dataset name
input_col = 'price'                                 # Dataset column of the independent variable
output_col = 'demand'                               # Dataset column of the dependent variable
result_col = 'revenue'                              # Result column of the objective function
store_id = "store_001"                              # Store ID used in scenario generation
test_size = 0.2                                     # Test size for train-test split

# Transformer parameters
sequence_length = 30                                # range: 10-30
num_layers = 4                                      # range: 2-6
size_layer = 128                                    # range: 64-256
embedded_size = 128                                 # range: 64-256
epochs = 500                                        # range: 200-500 w/ early stopping
dropout_rate = 0.2                                  # range: 0.1-0.3
learning_rate = 0.0001                              # range: 0.0001-0.001
batch_size = 64                                     # range: 32-128

#### 2. Load data

In [ ]:
# Load dataset

# AWS Consumer Electronics Dataset
df = pd.read_csv("datasets/aws_ce_dataset.csv", index_col="timestamp", parse_dates=True)

In [ ]:
# Dataset train-test split

unique_item_ids = df['item_id'].unique()
test_size = int(0.2*len(unique_item_ids))

# random items for testing
num_random_items = 100
random_item_ids = np.random.choice(unique_item_ids, num_random_items, replace=False)

# complete dataset
train_df = df[(df['item_id'].isin(random_item_ids)) & (df['store_id'] == store_id)]

In [ ]:
train_df

#### 3. Train model

In [ ]:
model_input = ImprovedTransformerForecasting(sequence_length=sequence_length, size_layer=size_layer, embedded_size=size_layer, output_size=1, dropout_rate=dropout_rate)
model_output = ImprovedTransformerForecasting(sequence_length=sequence_length, size_layer=size_layer, embedded_size=size_layer, output_size=1, dropout_rate=dropout_rate)

In [ ]:
def train_model(df, model, col, dataset_name="dataset"):
    """Training the transformer model on the training set"""

    train_prices = df[col].dropna().values
    model_file_path = f"trained-models/transformer-{dataset_name}-{col}.joblib"

    if os.path.exists(model_file_path):
        print(f"Trained model already exists -> loading model {model_file_path}")
        with open(model_file_path, 'rb') as file:
            model = pickle.load(file)
    else:
        print(f"Trained model not found -> training a new model {model_file_path}")
        # Train the model
        model.train(train_prices, batch_size, epochs, learning_rate)
        
        # Save the trained model
        joblib.dump({
            "model": model.model,
            "scaler": model.scaler,
            "sequence_length": model.sequence_length
        }, model_file_path)
        print("Model trained and saved successfully.")

    return model

In [ ]:
train_model(train_df, model_input, input_col, dataset_name)

In [ ]:
train_model(train_df, model_output, output_col, dataset_name)